Compute amplitude of Astrometric Signal:

Compare orbital phase of RV and astrometric orbits:

Compare the orbital angular momentum vector orientation with the star's:

Compute the flux ratio:

Compare the goodness of fit/chi2 of the 2-body fit to the 1-body fit:

Make corner plot, mass ratio plot, and period plot:

Update documentation :)